# Approche neuronale 

## Étape 1 : Chargement et Préparation des Données avec PyTorch

Trie des labels

In [5]:
import pandas as pd

# Remplacez 'path_to_file.txt' par le chemin réel vers votre fichier
file_path = 'Image/Projet IAI3/identity_CelebA.txt'

# Lire le fichier en supposant qu'il est séparé par des espaces
data = pd.read_csv(file_path, delim_whitespace=True, header=None, names=['filename', 'label'])

# Calculer le nombre de labels uniques
unique_labels_count = data['label'].nunique()

print(f'Il y a {unique_labels_count} labels uniques dans le fichier.')


Il y a 10177 labels uniques dans le fichier.


In [6]:
import pandas as pd
import json

# Remplacez 'your_file.txt' par le chemin d'accès à votre fichier réel
file_path = 'Image/Projet IAI3/identity_CelebA.txt'

# Lire le fichier
data = pd.read_csv(file_path, delim_whitespace=True, header=None, names=['filename', 'label'])

# Grouper les fichiers par label
grouped = data.groupby('label')['filename'].apply(list).to_dict()

# Créer une structure JSON
json_structure = {'data': grouped}

# Pour écrire dans un fichier JSON
with open('sorted_by_label.json', 'w') as json_file:
    json.dump(json_structure, json_file, indent=4)

# Afficher la structure JSON si nécessaire
#print(json.dumps(json_structure, indent=4))


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

# Définir les transformations, cela pourrait inclure la normalisation, le redimensionnement, etc.
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Charger les données de CelebA
# Vous devrez éventuellement ajuster les chemins et autres paramètres en fonction de votre organisation de fichiers
celeba_data = datasets.ImageFolder(root='Image/Projet IAI3/img_align_celeba', transform=transform)

# Diviser les données en ensembles d'entraînement, de validation et de test en utilisant les indices fournis par CelebA
train_indices = [...] # Liste des indices pour l'entraînement
val_indices = [...]   # Liste des indices pour la validation
test_indices = [...]  # Liste des indices pour le test

train_data = Subset(celeba_data, train_indices)
val_data = Subset(celeba_data, val_indices)
test_data = Subset(celeba_data, test_indices)

# Créer des DataLoaders pour itérer sur les données
batch_size = 32
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size)


## Étape 2 : Construire le Modèle avec PyTorch

In [2]:
import torch.nn as nn
import torch.nn.functional as F

# Définir un modèle simple CNN avec PyTorch
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1) # L'input doit correspondre au nombre de canaux de l'image
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 54 * 44, 512) # La taille doit correspondre à la sortie de la dernière couche MaxPool
        self.fc2 = nn.Linear(512, 1) # ou le nombre de classes pour la classification multi-classes

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 54 * 44) # Aplatir les sorties pour les connecter à la couche dense
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x)) # ou F.log_softmax pour la classification multi-classes
        return x

model = SimpleCNN()


# Etape 3 : Boucle d'entrainement
Cela inclut l'entraînement du modèle, le calcul de la perte, la mise à jour des poids du modèle, et la validation à la fin de chaque époque.

In [3]:
import torch.optim as optim

# Paramètres
epochs = 5  # Nombre d'époques
learning_rate = 0.001

# Définition de la fonction de perte et de l'optimiseur
criterion = nn.BCELoss()  # Pour la classification binaire
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Boucle d'entraînement
for epoch in range(epochs):
    model.train()  # Mettre le modèle en mode entraînement
    running_loss = 0.0

    for i, data in enumerate(train_loader, 0):
        # obtenir les entrées ; data est une liste de [inputs, labels]
        inputs, labels = data

        # zéro les gradients des paramètres
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()
        optimizer.step()

        # Imprimer les statistiques
        running_loss += loss.item()
        if i % 100 == 99:    # Imprimer toutes les 100 mini-batches
            print(f'Époque {epoch + 1}, Batch {i + 1}, Perte: {running_loss / 100:.4f}')
            running_loss = 0.0

    # Validation après chaque époque
    model.eval()  # Mettre le modèle en mode évaluation
    val_loss = 0.0
    val_steps = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            outputs = model(images)
            predicted = (outputs.squeeze() > 0.5).float()  # Seuil de 0.5 pour la classification binaire
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            loss = criterion(outputs.squeeze(), labels.float())
            val_loss += loss.item()
            val_steps += 1

    # Imprimer les statistiques de validation
    print(f'Validation Loss: {val_loss / val_steps:.4f}')
    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

print('Fin de l\'entraînement')


TypeError: list indices must be integers or slices, not ellipsis